In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
presi_csv = "Resources/documents_of_type_presidential_document_and_of_presidential_document_type_executive_order.csv"
execu_csv = "Resources/executive_orders.csv"
subtop_csv = "Resources/SubTopics.csv"
topco_csv = "Resources/TopicCode.csv"

In [3]:
presi_df = pd.read_csv(presi_csv)
presi_df = presi_df.astype({"executive_order_number": float})
presi_df.head()

,citation,document_number,end_page,executive_order_notes,executive_order_number,html_url,pdf_url,publication_date,signing_date,start_page,title
0,NaN,94-290,0,NaN,12890.0,https://www.federalregister.gov/documents/1994...,NaN,01/05/1994,12/30/1993,0,Amendment to Executive Order No. 12864
1,NaN,94-1531,0,"Revoked by: EO 13062, September 29, 1997",12891.0,https://www.federalregister.gov/documents/1994...,NaN,01/20/1994,01/15/1994,0,Advisory Committee on Human Radiation Experiments
2,NaN,94-1532,0,"Amends: EO 11063, November 20, 1962;;\n ; Rev...",12892.0,https://www.federalregister.gov/documents/1994...,NaN,01/20/1994,01/17/1994,0,Leadership and Coordination of Fair Housing in...
3,NaN,94-2261,0,"Revokes: EO 11063, November 20, 1962 (in part)...",12893.0,https://www.federalregister.gov/documents/1994...,NaN,01/31/1994,01/26/1994,0,Principles for Federal Infrastructure Investments
4,NaN,94-2267,0,"See: Memorandum, January 17, 1994",12894.0,https://www.federalregister.gov/documents/1994...,NaN,01/31/1994,01/26/1994,0,North Pacific Marine Science Organization


In [4]:
# presi_df.info()
# Create a filtered dataframe from specific columns
presi_cols = ["executive_order_number"]
presid_df = presi_df[presi_cols].copy()

# Rename the column headers
presid_df = presid_df.rename(columns={"executive_order_number": "Exec_Ord"})
presid_df.head()

,Exec_Ord
0,12890.0
1,12891.0
2,12892.0
3,12893.0
4,12894.0


In [5]:
execu_df = pd.read_csv(execu_csv)
execu_df['pres_party'] = execu_df['pres_party'].map({100: 'Democrat', 200: 'Republican'})
execu_cols = ["eo_number", "president", "pres_party", "year", "majortopic", "subtopic"]
execut_df = execu_df[execu_cols].copy()
execut_df = execut_df.rename(columns={"eo_number": "Exec_Ord", "president":"President", "pres_party": "Party", "year":"Year", "majortopic":"Topic", "subtopic": "Sub_Topic"})
execut_df.head()

,Exec_Ord,President,Party,Year,Topic,Sub_Topic
0,9538.0,TRUMAN,Democrat,1945,20,2004
1,9539.0,TRUMAN,Democrat,1945,19,1929
2,9540.0,TRUMAN,Democrat,1945,16,1610
3,9541.0,TRUMAN,Democrat,1945,20,2007
4,9542.0,TRUMAN,Democrat,1945,16,1610


In [6]:
# execu_df.info()
# Merge two dataframes using an outer join
merge_table1 = pd.merge(presid_df, execut_df, on="Exec_Ord", how="outer")
merge_table1.drop_duplicates(subset=['Exec_Ord'], keep=False)
merge_table1.head()

,Exec_Ord,President,Party,Year,Topic,Sub_Topic
0,12890.0,CLINTON,Democrat,1993.0,17.0,1709.0
1,12891.0,CLINTON,Democrat,1994.0,3.0,398.0
2,12892.0,CLINTON,Democrat,1994.0,14.0,1400.0
3,12893.0,CLINTON,Democrat,1994.0,10.0,1010.0
4,12894.0,CLINTON,Democrat,1994.0,19.0,1926.0


In [7]:
merge_table11 = merge_table1.dropna(how='all')
merge_table11.head()

,Exec_Ord,President,Party,Year,Topic,Sub_Topic
0,12890.0,CLINTON,Democrat,1993.0,17.0,1709.0
1,12891.0,CLINTON,Democrat,1994.0,3.0,398.0
2,12892.0,CLINTON,Democrat,1994.0,14.0,1400.0
3,12893.0,CLINTON,Democrat,1994.0,10.0,1010.0
4,12894.0,CLINTON,Democrat,1994.0,19.0,1926.0


In [8]:
merge_table11 = merge_table11.astype({"Sub_Topic": float})

In [9]:
subtop_df = pd.read_csv(subtop_csv, encoding='windows-1252')
subtop_df = subtop_df.rename(columns={"Code": "Sub_Topic", "SubTopic": "Sub_Topic_Title"})
subtop_df.head()

,Sub_Topic,Sub_Topic_Title,Description
0,100,General,Includes issues related to general domestic m...
1,101,Interest Rates,"Includes issues related to inflation, cost of..."
2,103,Unemployment Rate,Includes issues related to the unemployment r...
3,104,Monetary Policy,Includes issues related to the monetary polic...
4,105,National Budget,"Issues related to public debt, budgeting, and..."


In [10]:
# Merge two dataframes using an outer join
merge_table2 = pd.merge(subtop_df, merge_table11, on="Sub_Topic", how="outer")
merge_table2.drop_duplicates(subset=['Sub_Topic'], keep=False)
merge_table2.head()

,Sub_Topic,Sub_Topic_Title,Description,Exec_Ord,President,Party,Year,Topic
0,100.0,General,Includes issues related to general domestic m...,12912.0,CLINTON,Democrat,1994.0,1.0
1,100.0,General,Includes issues related to general domestic m...,13497.0,OBAMA,Democrat,2009.0,1.0
2,100.0,General,Includes issues related to general domestic m...,13499.0,OBAMA,Democrat,2009.0,1.0
3,100.0,General,Includes issues related to general domestic m...,13500.0,OBAMA,Democrat,2009.0,1.0
4,100.0,General,Includes issues related to general domestic m...,13501.0,OBAMA,Democrat,2009.0,1.0


In [11]:
topco_df = pd.read_csv(topco_csv)
topco_df = topco_df.rename(columns={"Code": "Topic", "Topic": "Topic_Title"})
topco_df = topco_df.astype({"Topic": float})
topco_df.head()

,Topic,Topic_Title
0,1.0,Macroeconomics
1,2.0,Civil Rights
2,3.0,Health
3,4.0,Agriculture
4,5.0,Labor


In [12]:
# Merge the two DataFrames together based on the Dates they share
merge_table3 = pd.merge(merge_table2, topco_df, on="Topic")
merge_table3.head()

,Sub_Topic,Sub_Topic_Title,Description,Exec_Ord,President,Party,Year,Topic,Topic_Title
0,100.0,General,Includes issues related to general domestic m...,12912.0,CLINTON,Democrat,1994.0,1.0,Macroeconomics
1,100.0,General,Includes issues related to general domestic m...,13497.0,OBAMA,Democrat,2009.0,1.0,Macroeconomics
2,100.0,General,Includes issues related to general domestic m...,13499.0,OBAMA,Democrat,2009.0,1.0,Macroeconomics
3,100.0,General,Includes issues related to general domestic m...,13500.0,OBAMA,Democrat,2009.0,1.0,Macroeconomics
4,100.0,General,Includes issues related to general domestic m...,13501.0,OBAMA,Democrat,2009.0,1.0,Macroeconomics


In [13]:
final_df = merge_table3.reindex_axis(['Exec_Ord',"Year","President","Party","Topic", "Sub_Topic","Topic_Title", "Sub_Topic_Title", "Description"], axis=1)
final_df
#.head()

/Users/adoteakpabie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


,Exec_Ord,Year,President,Party,Topic,Sub_Topic,Topic_Title,Sub_Topic_Title,Description
0,12912.0,1994.0,CLINTON,Democrat,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
1,13497.0,2009.0,OBAMA,Democrat,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
2,13499.0,2009.0,OBAMA,Democrat,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
3,13500.0,2009.0,OBAMA,Democrat,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
4,13501.0,2009.0,OBAMA,Democrat,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
5,13602.0,2012.0,OBAMA,Democrat,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
6,11453.0,1969.0,NIXON,Republican,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
7,11808.0,1974.0,FORD,Republican,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
8,11865.0,1975.0,FORD,Republican,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...
9,11903.0,1976.0,FORD,Republican,1.0,100.0,Macroeconomics,General,Includes issues related to general domestic m...


In [14]:
final_df = final_df.astype({"Year": int, "Topic": int, "Sub_Topic": int})
# Setting Executive order Number as Index
# final_df.set_index('Exec Ord#')
final_df.head()

,Exec_Ord,Year,President,Party,Topic,Sub_Topic,Topic_Title,Sub_Topic_Title,Description
0,12912.0,1994,CLINTON,Democrat,1,100,Macroeconomics,General,Includes issues related to general domestic m...
1,13497.0,2009,OBAMA,Democrat,1,100,Macroeconomics,General,Includes issues related to general domestic m...
2,13499.0,2009,OBAMA,Democrat,1,100,Macroeconomics,General,Includes issues related to general domestic m...
3,13500.0,2009,OBAMA,Democrat,1,100,Macroeconomics,General,Includes issues related to general domestic m...
4,13501.0,2009,OBAMA,Democrat,1,100,Macroeconomics,General,Includes issues related to general domestic m...


In [15]:
cols1 = ["Exec_Ord", "Year", "President", "Party", "Topic"]
executive_df = final_df[cols1].copy()
executive_df.drop_duplicates(subset=['Exec_Ord'], keep=False)
executive_df.set_index(executive_df.columns[0]).head()
# executive_df.head()

,Year,President,Party,Topic
Exec_Ord,,,,
12912.0,1994,CLINTON,Democrat,1
13497.0,2009,OBAMA,Democrat,1
13499.0,2009,OBAMA,Democrat,1
13500.0,2009,OBAMA,Democrat,1
13501.0,2009,OBAMA,Democrat,1


In [16]:
cols2 = ["Exec_Ord", "Topic", "Topic_Title", "Sub_Topic", "Sub_Topic_Title", "Description"]
description_df = final_df[cols2].copy()
description_df.drop_duplicates(subset=['Exec_Ord'], keep=False)
description_df.set_index(executive_df.columns[0]).head()
# description_df.head()

,Topic,Topic_Title,Sub_Topic,Sub_Topic_Title,Description
Exec_Ord,,,,,
12912.0,1,Macroeconomics,100,General,Includes issues related to general domestic m...
13497.0,1,Macroeconomics,100,General,Includes issues related to general domestic m...
13499.0,1,Macroeconomics,100,General,Includes issues related to general domestic m...
13500.0,1,Macroeconomics,100,General,Includes issues related to general domestic m...
13501.0,1,Macroeconomics,100,General,Includes issues related to general domestic m...


In [17]:
# Create a database connection
connection_string = "root:<insert password>@localhost/ETL_db"
engine = create_engine(f'mysql://{connection_string}')

In [18]:
# Confirm tables
engine.table_names()

['description', 'executive']

In [19]:
# Loading Data into Database

In [26]:
executive_df.to_sql(name='executive', con=engine, if_exists='replace', index=True)

In [25]:
description_df.to_sql('description', engine, if_exists='replace', index=True)